In [1]:
import pandas as pd
import requests

In [2]:
schools_url = "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/ricerca"
search_school_url =  "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/0CODE"
#search_school_url = "https://www.istruzione.provincia.tn.it/services/sei/api/v1/institutes/students/0CODE"
schools_trentino = pd.read_json(schools_url)
schools_trentino_covid_url = "https://www.vivoscuola.it/Coronavirus/NEWS/Numero-di-sezioni-classi-messe-in-quarantena-preventiva"
elenco_scuole="https://www.vivoscuola.it/facetsearch/datatable_search/istituto_plesso/viv_istituto|viv_plesso|institutetype|legalstate|dirigente_scolastico|direttore|coordinatore_pedagogico|viv_indirizzo|municipality|phone|fax|email|manageremail|officeemail|website|miurcode/extra_tipoistruzione_name____s|miurcode|legalstate|extra_adulti_name____s|extra_comune_name____s|typelocation/265/CSV?simpleQuery=&query=&extra_tipoistruzione_name____s=&miurcode=&legalstate=&extra_adulti_name____s=&show=10&extra_comune_name____s=&typelocation=&show=10"

In [3]:
def getStatusTipoScuola(valore):
    if (type(valore) == tuple):
        valore = valore[0]
    status = False
    if valore is not None:
        status = True
    return status

In [4]:
def getDataStudentsSchool(idschool):
    idschool = str(idschool)
    rdata = {}
    if (len(idschool) > 4):
        search_school = search_school_url.replace("CODE",idschool)
    else:
        search_school = search_school_url.replace("0CODE",idschool)
    json_data = requests.get(search_school).json()
    totalstudents =  json_data['totaleStudenti']
    totalclasses = json_data['totaleClassi']
    #kindinstruction = json_data['tipoIstituzione']
    primaria =  json_data['primaria'],
    secondaria1grado = json_data['secondaria1grado']
    secondaria2grado = json_data['secondaria2grado']
    formazioneprof = json_data['formazioneProf']
    #infanzia = json_data['infanzia']
    if (totalstudents == None):
        totalstudents = 0
    if (totalclasses == None):
        totalclasses = 0
    rdata['alunni'] = totalstudents
    rdata['classi'] = totalclasses
    #rdata['tipoistituzione'] = kindinstruction
    rdata['primaria'] = primaria
    rdata['secondaria1grado'] = secondaria1grado
    rdata['secondaria2grado'] = secondaria2grado
    rdata['formazioneprof'] = formazioneprof
    #rdata['infanzia'] = infanzia
    return rdata


In [5]:
schools_trentino['extra'] = schools_trentino['codiceProvinciale'].apply(getDataStudentsSchool)

In [6]:
schools_trentino = schools_trentino[~schools_trentino.idPadre.isnull()]
schools_trentino = pd.concat([schools_trentino, schools_trentino['extra'].apply(lambda x: pd.Series(x))], axis=1)
schools_trentino['alunni'] = schools_trentino['extra'].apply(lambda x: x['alunni'])
schools_trentino['classi'] = schools_trentino['extra'].apply(lambda x: x['classi'])
schools_trentino['primaria'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['primaria']))
schools_trentino['secondaria1grado'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['secondaria1grado']))
schools_trentino['secondaria2grado'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['secondaria2grado']))
schools_trentino['formazione_professionale'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['formazioneprof']))

In [7]:
del schools_trentino['istituzionePadre']
del schools_trentino['extra']
del schools_trentino['idPadre']
del schools_trentino['idobj']
del schools_trentino['formazioneprof']

In [8]:
schools_trentino[['codiceProvinciale','codiceMiur','denominazioneUfficiale','primaria','secondaria1grado','secondaria2grado','formazione_professionale','indirizzo','email','latitudeY','longitudeX','comune','alunni']].to_csv('alunni_scuole_trentino.csv',index=False)

In [74]:
schools_trentino_covid = pd.read_html(schools_trentino_covid_url,header=0)[0]

In [75]:
schools_trentino_covid.to_csv("alunni_scuole_trentino_quarantena.csv",index=False)

In [76]:
def changeDataRilevamento(v):
    if v.find("2021") == -1:
        if (v.split(" ")[1]) == "dicembre":
            v += " 2021"
        else:
            v += " 2022"
    return v


In [77]:
schools_trentino_covid['DATARILEVAMENTO'] = schools_trentino_covid['DATARILEVAMENTO'].apply(changeDataRilevamento)

In [78]:
schools_trentino_covid.rename(columns={
        'DATARILEVAMENTO':'data_rilevamento',
        'SCUOLAPRIMARIA':'num_classi_primaria',
        'SCUOLA SECONDARIADI I GRADO':'num_classi_secondaria_primo_grado',
        'SCUOLA SECONDARIADI II GRADO':'num_classi_secondaria_secondo_grado',
        'TAGESMUTTER':'tagesmutter',
        "NIDID'INFANZIA":'nido_infanzia',
        "SCUOLAINFANZIA":'scuola_infanzia'
        },inplace=True)

In [79]:
def changeDateString(day):
    day = day.split(" ")
    g = day[0]
    if len(g) <= 2:
        if len(g) == 1:
            g = "0" + g
        if len(day) == 2:
            daytmp = day[1].split("\xa0")
            m = daytmp[0]
            a = daytmp[1]
        else:
            m = day[1]
            a = day[2]
    else:
        g = day[0].split("\xa0")[0]
        m = day[0].split("\xa0")[1]
        a = day[1]
    m = m.lower()
    if m == "gennaio":
        m = "01"
    elif m == "febbraio":
        m = "02"
    elif m == "marzo":
        m = "03"
    elif m == "aprile":
        m = "04"
    elif m == "maggio":
        m = "05"
    elif m == "giugno":
        m = "06"
    elif m == "luglio":
        m = "07"
    elif m == "agosto":
        m = "08"
    elif m == "settembre":
        m = "09"
    elif m == "ottobre":
        m = "10"
    elif m == "novembre":
        m = "11"
    elif m == "dicembre":
        m = "12"
    day = g + "/" + m + "/" + a
    return day

In [80]:
schools_trentino_covid['data_rilevamento'] = schools_trentino_covid['data_rilevamento'].apply(changeDateString)

In [81]:
del schools_trentino_covid['TOTALE']

In [83]:
schools_trentino_covid.to_csv("classi_scuole_trentino_quarantena_rilevamento.csv",index=False)

In [84]:
totale_studenti_primaria = schools_trentino[schools_trentino['primaria']]['alunni'].sum()
totale_classi_primaria = schools_trentino[schools_trentino['primaria']]['classi'].sum()

In [86]:
totale_studenti_secondaria_primo_grado = schools_trentino[schools_trentino['secondaria1grado']]['alunni'].sum()
totale_classi_secondaria_primo_grado = schools_trentino[schools_trentino['secondaria1grado']]['classi'].sum()

In [87]:
totale_studenti_secondaria_secondo_grado = schools_trentino[schools_trentino['secondaria2grado']]['alunni'].sum() + schools_trentino[schools_trentino['formazione_professionale']]['alunni'].sum()
totale_classi_secondaria_secondo_grado = schools_trentino[schools_trentino['secondaria2grado']]['classi'].sum() + schools_trentino[schools_trentino['formazione_professionale']]['classi'].sum()

In [88]:
totale_alunni_per_scuole = pd.DataFrame([[totale_studenti_primaria,totale_studenti_secondaria_primo_grado,totale_studenti_secondaria_secondo_grado]],columns=['totale_alunni_primaria','totale_alunni_secondaria_primo_grado','totale_alunni_secondaria_secondo_grado'])
totale_classi_per_scuole = pd.DataFrame([[totale_classi_primaria,totale_classi_secondaria_primo_grado,totale_classi_secondaria_secondo_grado]],columns=['totale_classi_primaria','totale_classi_secondaria_primo_grado','totale_classi_secondaria_secondo_grado'])

In [89]:
totale_alunni_per_scuole.to_csv("totale_alunni_per_scuole_provincia_autonoma_trento.csv",index=False)
totale_classi_per_scuole.to_csv("totale_classi_per_scuole_provincia_autonoma_trento.csv",index=False)

In [90]:
def percentualeClassiQuaratena( classi,totale):
    p = (classi/totale)*100
    p = round(p,1)
    return p

In [92]:
schools_trentino_covid['percentuale_classi_primaria'] = schools_trentino_covid['num_classi_primaria'].apply(lambda x: percentualeClassiQuaratena(x,totale_classi_primaria))

In [93]:
schools_trentino_covid['percentuale_classi_secondaria_primo_grado']  = schools_trentino_covid['num_classi_secondaria_primo_grado'].apply(lambda x: percentualeClassiQuaratena(x,totale_classi_secondaria_primo_grado))

In [94]:
schools_trentino_covid['percentuale_classi_secondaria_grado']  = schools_trentino_covid['num_classi_secondaria_secondo_grado'].apply(lambda x: percentualeClassiQuaratena(x,totale_classi_secondaria_secondo_grado))

In [95]:
totale_alunni_scuole_primaria_primo_secondo_grado = totale_studenti_primaria + totale_studenti_secondaria_primo_grado + totale_studenti_secondaria_secondo_grado
totale_classi_scuole_primaria_primo_secondo_grado = totale_classi_primaria + totale_classi_secondaria_primo_grado + totale_classi_secondaria_secondo_grado

In [98]:
schools_trentino_covid

,data_rilevamento,tagesmutter,nido_infanzia,scuola_infanzia,num_classi_primaria,num_classi_secondaria_primo_grado,num_classi_secondaria_secondo_grado,percentuale_classi_primaria,percentuale_classi_secondaria_primo_grado,percentuale_classi_secondaria_grado
0,24/01/2022,4,29,108,112,69,143,6.4,7.0,7.5
1,17/01/2022,0,14,27,38,30,81,2.2,3.0,4.3
2,10/01/2022,0,3,0,0,0,0,0.0,0.0,0.0
3,27/12/2021,0,4,6,16,10,9,0.9,1.0,0.5
4,20/12/2021,0,3,5,21,10,8,1.2,1.0,0.4
5,13/12/2021,0,0,3,23,12,3,1.3,1.2,0.2
6,06/12/2021,0,0,5,13,12,3,0.7,1.2,0.2
7,29/11/2021,0,0,2,9,8,2,0.5,0.8,0.1
8,23/11/2021,0,0,2,8,4,4,0.5,0.4,0.2
9,15/11/2021,0,0,1,2,1,1,0.1,0.1,0.1


In [ ]:
schools_trentino_covid[['data_rilevamento','num_classi_primaria','percentuale_classi_primaria','num_classi_secondaria_primo_grado','percentuale_classi_secondaria_primo_grado','num_classi_secondaria_secondo_grado','percentuale_classi_secondaria_grado']].to_csv("percentuale_classi_quarantena_scuole_provincia_autonoma_trento.csv",index=False)